In [2]:
import urllib.request

opener = urllib.request.FancyURLopener({})
opener = urllib.request.FancyURLopener({})
url = "https://www.booli.se/sok/till-salu?page="
url = "https://www.booli.se/sok/slutpriser?page="
i = 1
f = opener.open(url + str(i))
html_content = f.read()

/var/folders/lv/g6xwdpyx3qd93zcp94tbn6b40000gn/T/ipykernel_10051/3823079157.py:3: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = urllib.request.FancyURLopener({})
/var/folders/lv/g6xwdpyx3qd93zcp94tbn6b40000gn/T/ipykernel_10051/3823079157.py:4: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = urllib.request.FancyURLopener({})


In [3]:
import requests
r = requests.get(url)

In [4]:
r.text

'<!DOCTYPE html><html lang="sv"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no"/><link rel="canonical" href="https://www.booli.se/sok/slutpriser"/><title>Slutpris för bostäder i Sverige – Booli</title><meta name="description" content="Slutpris för bostäder i Sverige hittar du på Booli. Vi visar även försäljningar, hur bostadsmarknaden utvecklas och hjälper dig att välja rätt mäklare."/><script type="application/ld+json">{"@context":"https://schema.org","@type":"ItemList","numberOfItems":2633266,"itemListElement":[{"@type":"ListItem","position":1,"url":"https://www.booli.se/annons/5564791"},{"@type":"ListItem","position":2,"url":"https://www.booli.se/bostad/1497521"},{"@type":"ListItem","position":3,"url":"https://www.booli.se/bostad/279037"},{"@type":"ListItem","position":4,"url":"https://www.booli.se/annons/5556692"},{"@type":"ListItem","position":5,"url":"https://www.booli.se/annons/5554833"},{"@type":"L

In [115]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
cards = soup.select('[class^=object-card__]')

In [116]:
data = {
    "street_name": [],
    "area": [],
    "city": [],
    "price": [],
    "size": [],
    "type": [],
}

In [117]:
cards_data = []
i = -1
card = []
for c in cards:
    if '<div class="object-card__header">' in str(c):
        continue
    if "<h3" in str(c):
        cards_data.append(card)
        card = [] 
        print("---")
    card.append(str(c))
    print(c)

---
<h3 class="object-card__heading"><a class="expanded-link no-underline hover:underline" href="/annons/5564791">Tellusgatan 3</a></h3>
<span class="object-card__date">2024-12-14</span>
<span class="object-card__preamble">Lägenhet · Planetstaden · Lund</span>
<span class="object-card__price">625 000 kr</span>
<ul class="object-card__data-list"><li>13 m²</li><li>1 rum</li><li>vån 1</li><li>48 100 kr/m²</li></ul>
<ul class="object-card__amenities"><li><div class="no-underline py-1 rounded w-fit flex-none tag--light inline-block rounded-sm px-1 text-label-4">Hiss</div></li></ul>
---
<h3 class="object-card__heading"><a class="expanded-link no-underline hover:underline" href="/bostad/279037">Volontärsgatan 3B</a></h3>
<span class="object-card__date">2024-12-14</span>
<span class="object-card__preamble">Lägenhet · Västra Berga · Helsingborg</span>
<span class="object-card__price">1 770 000 kr</span>
<ul class="object-card__data-list"><li>92 m²</li><li>4 rum</li><li>vån 1</li><li>19 200 kr/m

In [118]:
cards_data = cards_data[1:]
cards_data[1]

['<h3 class="object-card__heading"><a class="expanded-link no-underline hover:underline" href="/bostad/279037">Volontärsgatan 3B</a></h3>',
 '<span class="object-card__date">2024-12-14</span>',
 '<span class="object-card__preamble">Lägenhet · Västra Berga · Helsingborg</span>',
 '<span class="object-card__price">1\xa0770\xa0000\xa0kr</span>',
 '<ul class="object-card__data-list"><li>92\xa0m²</li><li>4\xa0rum</li><li>vån\xa01</li><li>19\xa0200\xa0kr/m²</li></ul>',
 '<ul class="object-card__amenities"><li><div class="no-underline py-1 rounded w-fit flex-none tag--light inline-block rounded-sm px-1 text-label-4">Balkong</div></li></ul>']

In [119]:
from pprint import pprint
cards_data[1][4].split(">")[2].split("<")[0].replace('\xa0', '').replace("m²", "")

'92'

In [120]:
import numpy as np
err_cnt = 0
for card in cards_data: 
    try:
        tags = card[2].split(">")[1].split("<")[0].split("·")
        assert len(tags) == 3
        price = card[3].split(">")[1].split("<")[0].split("·")[0].replace('\xa0', ' ').replace(" ", "").replace("kr", "")
        if price == "Utropsprissaknas":
            continue

        size = card[4].split(">")[2].split("<")[0].replace('\xa0', '').replace("m²", "").replace("½", ".5").replace("tomt", "").replace(" ", "")
        data["street_name"].append(str(card[0].split(">")[2].split("<")[0]))
        data["type"].append(tags[0])
        data["area"].append(tags[1])
        data["city"].append(tags[2])
        data["price"].append(float(price))
        data["size"].append(float(size))
    except: 
        print(card)
        print(size)
        err_cnt += 1 
        
err_cnt, len(cards_data)

['<h3 class="object-card__heading"><a class="expanded-link no-underline hover:underline" href="/bostad/2487351">Erstorp 502</a></h3>', '<span class="object-card__date">2024-12-14</span>', '<span class="object-card__preamble">Villa · Finspång</span>', '<span class="object-card__price">990\xa0000\xa0kr</span>', '<ul class="object-card__data-list"><li>94\xa0m²</li><li>3\xa0rum</li><li>2 160\xa0m²\xa0tomt</li><li>10\xa0500\xa0kr/m²</li></ul>', '<ul class="object-card__amenities"><li><div class="no-underline py-1 rounded w-fit flex-none tag--light inline-block rounded-sm px-1 text-label-4">Eldstad</div></li><li><div class="no-underline py-1 rounded w-fit flex-none tag--light inline-block rounded-sm px-1 text-label-4">Uteplats</div></li></ul>']
66
['<h3 class="object-card__heading"><a class="expanded-link no-underline hover:underline" href="/bostad/3445924">Läraregatan 19</a></h3>', '<span class="object-card__date">2024-12-14</span>', '<span class="object-card__preamble">Villa · Ulricehamn</

(2, 34)

In [121]:
for k, v in data.items():
    print(f"{k}: {len(v)}")

street_name: 32
area: 32
city: 32
price: 32
size: 32
type: 32


In [122]:
import pandas as pd 

df = pd.DataFrame(data)
df.head()

,street_name,area,city,price,size,type
0,Tellusgatan 3,Planetstaden,Lund,625000.0,13.0,Lägenhet
1,Volontärsgatan 3B,Västra Berga,Helsingborg,1770000.0,92.0,Lägenhet
2,Tegelbruksvägen 7F,Ekerö Centrum,Ekerö,2550000.0,61.0,Lägenhet
3,Sandelsgatan 35,Gärdet,Stockholm,7800000.0,65.0,Lägenhet
4,Furudal Saharavägen 17,Furudal,Rättvik,510000.0,78.0,Villa
